# TensorFlow v2.16.1 - Apple Silicon

**Apple Silicon users:** If you previously installed TensorFlow using `pip install tensorflow-macos`, please update your installation method. Use `pip install tensorflow` from now on.

> **Note:** `pip install tensorflow-metal` is still required to get access to GPU.
> <br>
> ```python
> import tensorflow as tf
> 
> for device in ['CPU', 'GPU']:
>     print(tf.config.list_physical_devices(device))
> 
> [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
> [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]  # pip install tensorflow-metal
> ```

References:
1. [Tensorflow 2.16 Release Notes](https://github.com/tensorflow/tensorflow/releases)
2. [Get started with tensorflow-metal](https://developer.apple.com/metal/tensorflow-plugin/)

Baseline CPU vs GPU:
1. Kaggle: [Tensorflow 2.15 CPU/GPU Comparison](https://www.kaggle.com/code/gdataranger/tensorflow-2-16-cpu-gpu-comparison)
2. Colab: [Tensorflow 2.15 CPU/GPU Comparison](https://colab.research.google.com/drive/1FyQ1VqYjsGCGtUcNwM0FkGRPFnTIQrR1)

## Setup

> TODO: [TF 2.16.1 and Apple Silicon GPU support not working](https://github.com/tensorflow/tensorflow/issues/63854)
> We need to install **`tf_keras~=2.16`** and then set **`os.environ['TF_USE_LEGACY_KERAS'] = '1'`** in our kernel to have access to the GPU.

Start from a fresh virtual environment (venv) in order to ensure we have the correct requirements
installed for `v2.16.1`, as the differences from `v2.15.0` to `v2.15.1` to `v2.16.1`
have vastly different requirements.
<br>

```zsh
~ % cd ~/.virtualenvs
.virtualenvs % python3 -m venv venv-tf16
.virtualenvs % source venv-tf16/bin/activate
(venv-tf16) .virtualenvs % pip install --upgrade pip
(venv-tf16) .virtualenvs % pip list

Package    Version
---------- -------
pip        24.0
setuptools 65.5.0

(venv-tf16) .virtualenvs % pip install tensorflow~=2.16 tf_keras~=2.16
(venv-tf16) .virtualenvs % pip install tensorflow-metal
(venv-tf16) .virtualenvs % pip freeze > requirements_tf216.txt
(venv-tf16) .virtualenvs % pip list | grep -E 'keras|tensorflow'
keras                        3.0.5
tensorflow                   2.16.1
tensorflow-io-gcs-filesystem 0.36.0
tensorflow-metal             1.1.0
tf_keras                     2.16.0

(venv-tf16) ~ % jupyter notebook --port=8890
```

Now let's ensure we have this virtual environment available as a Jupyter kernel.

```zsh
(venv-tf16) .virtualenvs % pip install ipykernel
(venv-tf16) .virtualenvs % python3 -m ipykernel install --user --name=venv-tf16 --display-name="Python (TensorFlow 2.16)"
Installed kernelspec venv-tf16 in /Users/marksusol/Library/Jupyter/kernels/venv-tf16

(venv-tf16) .virtualenvs % jupyter kernelspec list
Available kernels:
ir           /Users/marksusol/Library/Jupyter/kernels/ir
venv-tf15    /Users/marksusol/Library/Jupyter/kernels/venv-tf15
venv-tf16    /Users/marksusol/Library/Jupyter/kernels/venv-tf16
python3      /Library/Frameworks/Python.framework/Versions/3.11/share/jupyter/kernels/python3
```

## JetBrain DataSpell

Ensure that **DataSpell** is correctly configured to use the `venv-tf16` virtual environment and properly loads the python/tensorflow versions.

![tensorflow-dataspell-tf15](../images/tensorflow-dataspell-tf16.png)

## Tensorflow

In [1]:
import sys
import tensorflow as tf

# Ensure the notebook runs on the correct environment in DataSpell.
assert sys.executable == '/Users/marksusol/.virtualenvs/venv-tf16/bin/python3'
assert tf.__version__ == '2.16.1'

print(f'System: {sys.executable}')
print(f'TensorFlow: {tf.__version__}')

System: /Users/marksusol/.virtualenvs/venv-tf16/bin/python3
TensorFlow: 2.16.1


### GPU Test

In [2]:
%%time
import os
# When = 3, the messages (1 - informational(I), 2 - warnings(W) and 3- errors(E)) will not be logged during code execution.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import tensorflow as tf
# Ensure we see the GPU in device list.
print('Visible Devices: ', tf.config.get_visible_devices())

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=1, batch_size=128)

Visible Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
391/391 [==============================] - 58s 94ms/step - loss: 4.2792 - accuracy: 0.1048
CPU times: user 28.5 s, sys: 6.92 s, total: 35.4 s
Wall time: 59.8 s


### CPU Testing

This test can be run by itself after restarting the runtime to ensure we don't run into the `cannot modify virtual devices once initialized.` message. We're not able to 'renable' the GPU device once disabled.

**Note:** You may have your jupyter notebook running on a different port, for this notebook we are running the notebook on `--port=8890` by default in **DataSpell**.


In [1]:
%%capture
!jupyter notebook stop 8890

In [2]:
%%time

import tensorflow as tf
# Removes GPU from list, i.e. []
tf.config.set_visible_devices([], 'GPU')
print('Visible Devices: ', tf.config.get_visible_devices())

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=1, batch_size=128)

Visible Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
391/391 ━━━━━━━━━━━━━━━━━━━━ 3090s 8s/step - accuracy: 0.0611 - loss: 4.7813
CPU times: user 20min 54s, sys: 3min 6s, total: 24min
Wall time: 51min 30s
